In [1]:
import pandas as pd

df = pd.read_excel(r"C:\Users\CAP-016\Downloads\0.All Master Data (1).xlsx", sheet_name="O2C_Sample (Bots 1-20) sale reg")

In [2]:
df.head()

Customer   SO_No    SO_Date  SLA_Dispatch_Days Delivery_No  \
0   Customer_8  SO1711 2024-01-17                 11     DEL5412   
1   Customer_8  SO1711 2024-05-06                 18     DEL9070   
2  Customer_10  SO1541 2024-02-17                 18     DEL7655   
3   Customer_9  SO2753 2024-06-21                 15     DEL6070   
4  Customer_10  SO4110 2024-05-03                  6     DEL7082   

  Delivery_Date  Gap b/w SO & Delivery Date Invoice_No Invoice_Date  SO_Qty  \
0    2024-01-25                           8   INV15966   2024-01-26      75   
1    2024-05-19                          13   INV68042   2024-01-26      12   
2    2024-03-03                          15   INV74775   2024-03-03      61   
3    2024-07-04                          13   INV34019   2024-07-07      50   
4    2024-05-03                           0   INV74614   2024-05-07      79   

   ...  Approved_Rate  Deviation (%)  GST_Rate%  CGST%  SGST% IGST%  \
0  ...            124           0.80         18      9      9     0   
1  ...            426           0.93         18      9      9     0   
2  ...            455           5.43         18      9      9     0   
3  ...            252           8.46         12      6      6     0   
4  ...            318           0.63         18      9      9     0   

   CGST_Amount  SGST_Amount IGST_Amount Total_Invoice_Amount_with_GST  
0   832.815000   832.815000           0                  10919.130000  
1   455.801256   455.801256           0                   5976.060912  
2  2362.311315  2362.311315           0                  30972.526130  
3   692.042400   692.042400           0                  12918.124800  
4  2246.735826  2246.735826           0                  29457.203052  

[5 rows x 30 columns]

In [5]:
# 24 O2C -  SO dump

import pandas as pd
import numpy as np

def sales_without_so(df):
    """
    Bot: Sales without SO
    Rule:
    - Pick rows where SO_No is missing (NaN or empty)
      AND Invoice_No is present (not NaN & not empty)
    - Add column 'Exception_Noted' with:
        'Invoice without SO ID' or 'Ok'
    """

    # Ensure required columns exist
    if "SO_No" not in df.columns or "Invoice_No" not in df.columns:
        return pd.DataFrame()

    # Clean values for consistent checks
    df["SO_No"] = df["SO_No"].astype(str).str.strip()
    df["Invoice_No"] = df["Invoice_No"].astype(str).str.strip()

    # Condition for missing SO but invoice present
    condition = (
        ((df["SO_No"] == "") | (df["SO_No"].isna()) | (df["SO_No"].str.lower() == "nan"))
        &
        ((df["Invoice_No"] != "") & (df["Invoice_No"].str.lower() != "nan"))
    )

    # Add exception column using np.where
    df["Exception_Noted"] = np.where(
        condition,
        "Invoice without SO ID",
        "Ok"
    )

    # Return only exception rows (same style as other bots)
    return df[df["Exception_Noted"] == "Invoice without SO ID"].reset_index(drop=True)


sales_without_so(df).to_excel(r"C:\Users\CAP-016\Downloads\Audit Bots Output\O2C Output\Bot 24 (Sales without SO).xlsx")


In [6]:
# 29 O2C -  SO dump

import pandas as pd
import numpy as np

def multiple_invoices_for_so(df):
    """
    Bot: Multiple Invoices for Same SO
    Step:
    - Count invoices per SO_No
    - Mark 'Multiple Invoices' if count > 1, else 'Standard'
    - Return only rows where Exception_Noted == 'Multiple Invoices'
    """

    # Ensure required columns exist
    if "SO_No" not in df.columns or "Invoice_No" not in df.columns:
        return pd.DataFrame()

    # Normalize values to avoid issues (strip spaces, convert nan to "")
    df["SO_No"] = df["SO_No"].astype(str).str.strip()
    df["Invoice_No"] = df["Invoice_No"].astype(str).str.strip()

    # Count invoice per SO_No
    invoice_counts = (
        df[df["Invoice_No"] != ""]
        .groupby("SO_No")["Invoice_No"]
        .transform("count")
    )

    # Add Exception_Noted column
    df["Exception_Noted"] = np.where(
        invoice_counts > 1,
        "Multiple Invoices",
        "Standard"
    )

    # Return only problematic rows
    return df[df["Exception_Noted"] == "Multiple Invoices"].reset_index(drop=True)

multiple_invoices_for_so(df).to_excel(r"C:\Users\CAP-016\Downloads\Audit Bots Output\O2C Output\Bot 29 (Multiple Invoices per Order).xlsx")


In [7]:
# 32 O2C -  SO dump


import pandas as pd
import numpy as np

def zero_or_missing_invoice_amount(df):
    """
    Bot: Zero or Missing Invoice Amount
    Step:
    - Pick rows where Invoice_Amount == 0 or '' or NaN
    - Add Exception_Noted column
    - Return only problematic rows
    """

    # Ensure column exists
    if "Invoice_Amount" not in df.columns:
        return pd.DataFrame()

    # Convert to string for blank checks & also keep numeric version
    df["Invoice_Amount_str"] = df["Invoice_Amount"].astype(str).str.strip()

    # Condition for zero / missing
    condition = (
        (df["Invoice_Amount"].fillna(0) == 0) |  # numeric zero or NaN treated as zero
        (df["Invoice_Amount_str"] == "") |       # empty string
        (df["Invoice_Amount_str"].str.lower() == "nan")  # 'nan' as string
    )

    # Add Exception_Noted column
    df["Exception_Noted"] = np.where(
        condition,
        "Zero / Missing Amount",
        "Ok"
    )

    # Return only exceptions
    return df[df["Exception_Noted"] == "Zero / Missing Amount"].reset_index(drop=True)

zero_or_missing_invoice_amount(df).to_excel(r"C:\Users\CAP-016\Downloads\Audit Bots Output\O2C Output\Bot 32 (Zero-priced Invoices).xlsx")

In [9]:
# 39 O2C -  SO dump


import pandas as pd
import numpy as np

def excessive_small_value_sales(df, amount_threshold = 10000, percent_threshold=0.1):
    """
    Bot: Excessive Small-Value Sales
    - Identify small-value sales (Invoice_Amount <= amount_threshold)
    - Compute % of small-value sales
    - Mark ONLY small-value rows:
         -> 'Alert' if % >= percent_threshold
         -> 'Standard' if % < percent_threshold
    - Return only small-value sales rows
    """

    if "Taxable_Amount" not in df.columns:
        return pd.DataFrame()

    # Convert to numeric
    df["Taxable_Amount_clean"] = pd.to_numeric(df["Taxable_Amount"], errors="coerce")

    # Identify small-value sales
    df["Is_Small_Value"] = df["Taxable_Amount_clean"] <= amount_threshold

    # Calculate small-value percentage
    total_rows = len(df)
    small_value_count = df["Is_Small_Value"].sum()

    if total_rows == 0:
        return pd.DataFrame()

    small_value_percent = (small_value_count / total_rows) * 100

    # Decide final label for only small value rows
    label_for_small_value = (
        "Alert" if small_value_percent >= percent_threshold else "Standard"
    )

    # Apply label ONLY to small-value rows
    df.loc[df["Is_Small_Value"], "Exception_Noted"] = label_for_small_value

    # Return only the small-value rows
    return df[df["Is_Small_Value"]].drop(columns=["Is_Small_Value"]).reset_index(drop=True)

excessive_small_value_sales(df).to_excel(r"C:\Users\CAP-016\Downloads\Audit Bots Output\O2C Output\Bot 39 (Excessive Small-value Sales).xlsx")

In [16]:
import numpy as np

df1 = pd.read_excel(r"C:\Users\CAP-016\Downloads\0.All Master Data (1).xlsx", sheet_name="Employee Master")

In [17]:
df1.head(10)

Employee_ID Employee_Name Employee_Name.1  Join_Date  Exit_Date  \
0        E001         Emp_1    Aarav Sharma 2019-01-02 2024-03-20   
1        E002         Emp_2     Priya Verma 2018-03-16 2024-05-19   
2        E003         Emp_3     Rohan Gupta 2019-02-04 2024-05-20   
3        E004         Emp_4     Neha Kapoor 2019-01-19        NaT   
4        E005         Emp_5    Vikram Singh 2023-11-29        NaT   
5        E006         Emp_6    Anjali Mehta 2023-02-10 2024-03-14   
6        E007         Emp_7     Arjun Reddy 2024-06-26 2024-05-08   
7        E008         Emp_8      Sneha Nair 2019-05-17        NaT   
8        E009         Emp_9  Karan Malhotra 2024-04-20 2024-04-07   
9        E010        Emp_10     Meera Joshi 2024-04-11 2024-06-24   

  Access_End_Date Designation   Department Gender    Status  ... Approved By  \
0      2024-03-20     Manager           HR      M  Inactive  ...        E001   
1      2024-05-21   Executive           HR      F  Inactive  ...        E001   
2      2024-05-20  Supervisor  Maintenance      M  Inactive  ...        E001   
3             NaT     Manager   Production      F    Active  ...        E001   
4             NaT  Supervisor      Finance      M    Active  ...        E001   
5      2024-03-14     Manager           HR      F  Inactive  ...        E001   
6      2024-05-08  Supervisor        Sales      M  Inactive  ...        E011   
7             NaT    Engineer           IT      F    Active  ...        E001   
8      2024-04-07  Supervisor      Finance      M  Inactive  ...        E001   
9      2024-06-24    Operator   Production      F  Inactive  ...        E001   

         DOB       Phone  Bank_Account        IFSC      PAN_No Monthly_Salary  \
0 1990-06-15  9876543210      AC791433  IFSC000000  AAAAA1234A   65833.333333   
1 1994-09-20  9876501234      AC987147  IFSC000001  BBBBB2345B   44573.333333   
2 1996-02-12  9823456789      AC987147  IFSC000002  CCCCC3456C   71833.333333   
3 1989-03-08  9811122233      AC987149  IFSC000003  DDDDD4567D   34560.833333   
4 1987-07-21  9898989898      AC910518  IFSC000004  EEEEE5678E  117858.333333   
5 1997-05-02  9876009876      AC443006  IFSC000005  FFFFF6789F   34893.333333   
6 1995-12-12  9812233445      AC388622  IFSC000006  AAAAA1234A   60408.333333   
7 1993-10-10  9823456123      AC388622  IFSC000007  HHHHH8901H   39193.333333   
8 1986-04-14  9811456789      AC388624  IFSC000008  IIIII9012I   36748.333333   
9 1999-01-25  9876549090      AC791433  IFSC000009  JJJJJ0123J   32643.333333   

   Creator_ID Approved_By Exception_Flag  
0        E002        E006             No  
1        E002        E006             No  
2        E002        E001            Yes  
3        E002        E013            Yes  
4        E002        E006            Yes  
5        E002        E006             No  
6        E002        E001            Yes  
7        E002        E006            Yes  
8        E002        E013            Yes  
9        E026        E001            Yes  

[10 rows x 21 columns]

In [18]:
# 44 H2R -  Employee Master

import pandas as pd
import numpy as np

def duplicate_employees(df):
    """
    Bot: Duplicate Employees
    Rules:
    - Rows where Bank_Account OR PAN_No is duplicated
    - Exception_Noted shows type of duplicate
    - Duplicate_Group identifies duplicate grouping
    - Return only duplicate rows
    """

    # Validate required columns
    required_cols = ["Bank_Account", "PAN_No"]
    for col in required_cols:
        if col not in df.columns:
            return pd.DataFrame()

    # Normalize values
    df["Bank_Account"] = df["Bank_Account"].astype(str).str.strip()
    df["PAN_No"] = df["PAN_No"].astype(str).str.strip()

    # Identify duplicates
    df["Dup_Bank"] = df.duplicated("Bank_Account", keep=False) & (df["Bank_Account"] != "")
    df["Dup_PAN"] = df.duplicated("PAN_No", keep=False) & (df["PAN_No"] != "")

    # Build exception text
    df["Exception_Noted"] = np.where(
        df["Dup_Bank"] & df["Dup_PAN"],
        "Duplicate Bank & PAN",
        np.where(
            df["Dup_Bank"],
            "Duplicate Bank Account",
            np.where(df["Dup_PAN"], "Duplicate PAN", "")
        )
    )

    # Build duplicate grouping key
    df["Duplicate_Group"] = np.where(
        df["Dup_Bank"] & df["Dup_PAN"],
        "BANKPAN_" + df["Bank_Account"] + "_" + df["PAN_No"],
        np.where(
            df["Dup_Bank"],
            "BANK_" + df["Bank_Account"],
            np.where(df["Dup_PAN"], "PAN_" + df["PAN_No"], "")
        )
    )

    # Return only duplicate rows
    return df[df["Exception_Noted"] != ""].reset_index(drop=True)

duplicate_employees(df1).to_excel(r"C:\Users\CAP-016\Downloads\Audit Bots Output\H2R Output\Bot 44 (Duplicate Employees).xlsx")